# Advent of Code Day 10

For day 10, the goal was to implement a custom hashing algorithm called a knot hash.  The rules were included in the problem but basically you step through a list of integers in range [0..255] and reverse portions of it based on some rules.  In part one, once the reversals are complete the question is the result of multiplying the first two values.  In part two, it's actually to use part one's algorithm as one round of 16 total plus to apply a compression and hexadecimal translation thereafter.  I found this problem to be one the hardest one so far.  That surprised me since on paper, the problem is easy enough to understand.  The logic to reverse the input when the reversible section wrapped around the back of the string and into the front was surprisngly tricky for me.  My solution isn't the most efficient, but it does the trick.  Ultimately, I decided to temporarily rotate the input so that the reversible segment is at the very front of the string.  This made reversing it easy to deal with.  Then, I rotated it back to the original place.  I tried reversing it as part of a list concatenation but couldn't get that to work properly.  The base case is when you're reversing a section that fits within the string with no wrapping and that does rely on concatenation and slicing with no rotation.  I like the use of KnotHash as a class here though there's some parts that I'm uncertain are as Pythonic as they could be.  I very much like how Python is forcing me to think about encapsulation and class design, but it's hard to fight the long-held urges learned in other languages.  

In [ ]:
from utils import read_input

### Rotations

For reversing sections of the knot within KnotHash, I ultimately settled on a solution for rotating the input so place the reversible section at the front, then rotating it back after reversal.  I decided to keep these out of the class since they're somewhat more general purpose than that.

In [ ]:
def rotate_left(L, by):
    if by > len(L):
        by = by % len(L)
        
    return L[by:] + L[:by] if by > 0 else rotate_right(L, -by)

def rotate_right(L, by):
    if by > len(L):
        by = by % len(L)
        
    return L[-by:] + L[:-by] if by > 0 else rotate_left(L, -by)   

### KnotHash

I didn't start off having this class but refactored to it in part 2 due to the requirement to retain current index and skip size across rounds.  This class is responsible for the knot hash algorithm outlined in the instructions, which includes part 1 (hash()) and part 2 (hex_hash()).  Current index and skip size are treated as "public" variables and thus suggest the user can adjust them.  That's true, though perhaps misleading as I'm not certain that's really my intent.  I'll have to think about that more and maybe move them to "private" and put them in the constructor.  I'm using "public" and "private" here in the loose sense since Python of course doesn't support those concepts the way other languages do, but do so by naming convention.  I did mark the knot itself as private, after much internal debate.  Hash() returns the value of knot after it's complete, which is needed by part 1 (though could have been provided another way, such as knot being public).  

In [ ]:
class KnotHash(object):
    
    def __init__(self, knot, lengths):
        self.current_index = 0
        self.skip_size = 0
        self._knot = knot
        self.lengths = lengths
    
    def hash(self, rounds):
        self._do_hash_rounds(rounds)
        
        return self._knot
    
    def hex_hash(self, rounds, hash_step):
        
        self.hash(rounds)
        
        return self._compute_hexadecimal_hash(hash_step)
    
    def _do_hash_rounds(self, rounds):
        for _ in range(0, rounds):
            for i, length in enumerate(self.lengths):
                self._knot = self._reverse_section(length)
                self.current_index = self._compute_next_index(length)
                self.skip_size +=1                    
    
    def _compute_next_index(self, length):
        return (self.current_index + length + self.skip_size) % len(self._knot)
    
    def _reverse_section(self, length):     
    
        if self.current_index + length <= len(self._knot):
            return self._knot[:self.current_index] + list(reversed(self._knot[self.current_index:self.current_index+length])) + \
                   self._knot[self.current_index+length:]        
        else:              
            rotated = rotate_left(self._knot, self.current_index)
            
            after_reverse = list(reversed(rotated[:length]))
            
            combined = after_reverse + rotated[length:]
            
            rotated_back = rotate_right(combined, self.current_index)
            
            return rotated_back
    
    def _compute_hexadecimal_hash(self, hash_step):           
        dense_hash = [None] * hash_step
        for index, hash_index in enumerate(range(0, 256, hash_step)):
            dense_hash[index] = self._sparse_hash(hash_index, hash_step)
    
        return ''.join([format(d, '02x') for d in dense_hash])
    
    def _sparse_hash(self, hash_index, hash_step):       
        return reduce(lambda x, y: x^y, self._knot[hash_index:hash_index+hash_step])
            

### Solve Part One

Initially part one was solved with free-standing functions to compute the knot hash as described by the problem.  However, part 2 required more extensive computations and it made sense to refactor to a KnotHash class instead. Part one still works correctly, however, but it's important to note that the the solution at hand now is not the original one.  Too, to support both parts, KnotHash has both hash() and hex_hash() methods to support the two situations.  I contemplated a couple options for this.  One: it appears to be Pythonic to have the user just reference a class property if desired so hash() would still operate on class members and I would read the knot variable out of it instead of having hash() return it.  This would allow a single hash() method to still be possible.  A second option I contemplated was to pass in the output mode to hash() and have it determine its output based on that.  That could be a Pythonic solution, but it still felt a little weird to me. Perhaps in time I'll get over that and realize it's for the best.  I'm already liking so much of the Python style where classes are concerned.  The "everyone is a consenting adult" model really has me approaching things a lot differently. 

In [ ]:
def solve_part_one():
    lengths = [int(i) for i in read_input('Input/day10.txt')[0].split(',')]
   
    knot_hash = KnotHash(range(0, 256), lengths)
    
    hashed = knot_hash.hash(1)
    
    print 'Multiplication of first two elements is {} * {} = {}'.format(hashed[0], hashed[1], hashed[0] * hashed[1])

### Solve Part Two

Part 2 is an extension of part 1.  We learn for part 2 that part 1 is simply one step in a multi-step algorithm to compute the knot hash.  Specifically, we need to alter our understanding of the prior input (to treat each character as its ASCII value and not treat them as length values directly), then we need to run the knot hash 64 times with the same lengths but preserving the index and skip size across runs.  After doing so, we have to compress the input down to a 16-dimension vector and form the hexadecimal number by pairing up dimensions (1 to 2, 3 to 4, etc).  It was part two that actually drove me to pull out a KnotHash class.  Before this, I'd made do with doing that as functions, but now needing to preserve index and skip size, it made more sense to have a class own that state.  To still support both parts 1 and 2, hash() simply performs the knot algorithm and then returns the knot while hex_hash() does the knot plus compresses it down to a hexadecimal expression.

In [ ]:
def solve_part_two():
    lengths = [ord(i) for i in read_input('Input/day10.txt')[0]] + [17, 31, 73, 47, 23]
      
    knot_hash = KnotHash(range(0, 256), lengths)
    
    hashed = knot_hash.hex_hash(64, 16)
    
    print hashed    

In [ ]:
solve_part_one()
solve_part_two()